## Single files

This is a dummy example file we use for our tests:

In [1]:
!h5ls fxe_control_example.h5

CONTROL                  Group
INDEX                    Group
INSTRUMENT               Group
METADATA                 Group
RUN                      Group


In [2]:
from karabo_data import H5File
f = H5File('fxe_control_example.h5')

In [3]:
f.control_devices

{'FXE_XAD_GEC/CAM/CAMERA',
 'SA1_XTD2_XGM/DOOCS/MAIN',
 'SPB_XTD9_XGM/DOOCS/MAIN'}

In [4]:
f.instrument_device_channels

{'FXE_XAD_GEC/CAM/CAMERA:daqOutput',
 'SA1_XTD2_XGM/DOOCS/MAIN:output',
 'SPB_XTD9_XGM/DOOCS/MAIN:output'}

In [5]:
print(f.train_ids[:10])

[10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009]


### Get data by train

In [6]:
for tid, data in f.trains():
    print("Processing train", tid)
    print("beam iyPos:", data['SA1_XTD2_XGM/DOOCS/MAIN']['beamPosition.iyPos.value'])
    
    break

Processing train 10000
beam iyPos: 0.0


In [7]:
tid, data = f.train_from_id(10005)
data

{'FXE_XAD_GEC/CAM/CAMERA': {'acquisitionTime.timestamp': 0,
  'acquisitionTime.value': 0.0,
  'binningX.timestamp': 0,
  'binningX.value': 0,
  'binningY.timestamp': 0,
  'binningY.value': 0,
  'coolingLevel.timestamp': 0,
  'coolingLevel.value': 0,
  'cropLines.timestamp': 0,
  'cropLines.value': 0,
  'enableBiasCorrection.timestamp': 0,
  'enableBiasCorrection.value': 0,
  'enableBurstMode.timestamp': 0,
  'enableBurstMode.value': 0,
  'enableCooling.timestamp': 0,
  'enableCooling.value': 0,
  'enableCropMode.timestamp': 0,
  'enableCropMode.value': 0,
  'enableExtTrigger.timestamp': 0,
  'enableExtTrigger.value': 0,
  'enableShutter.timestamp': 0,
  'enableShutter.value': 0,
  'enableSync.timestamp': 0,
  'enableSync.value': 0,
  'exposureTime.timestamp': 0,
  'exposureTime.value': 0,
  'firmwareVersion.timestamp': 0,
  'firmwareVersion.value': 0,
  'metadata': {'source': 'FXE_XAD_GEC/CAM/CAMERA',
   'timestamp': {'frac': 578696, 'sec': 1525257587, 'tid': 10005}},
  'modelId.timest

In [8]:
tid, data = f.train_from_index(5)

### Get tables of data with pandas

This is a work in progress on the *pandas* branch in git; it doesn't yet work on master.

In [9]:
ixPos = f.get_series('SA1_XTD2_XGM/DOOCS/MAIN', 'beamPosition.ixPos.value')
ixPos.tail(10)

trainId
10390    0.0
10391    0.0
10392    0.0
10393    0.0
10394    0.0
10395    0.0
10396    0.0
10397    0.0
10398    0.0
10399    0.0
Name: SA1_XTD2_XGM/DOOCS/MAIN/beamPosition.ixPos, dtype: float32

In [10]:
# Select by train ID
ixPos.loc[:10010]

trainId
10000    0.0
10001    0.0
10002    0.0
10003    0.0
10004    0.0
10005    0.0
10006    0.0
10007    0.0
10008    0.0
10009    0.0
10010    0.0
Name: SA1_XTD2_XGM/DOOCS/MAIN/beamPosition.ixPos, dtype: float32

In [11]:
# The format of this is likely to change
df = f.get_dataframe()
df.head()

,SPB_XTD9_XGM/DOOCS/MAIN/beamPosition.ixPos,SPB_XTD9_XGM/DOOCS/MAIN/beamPosition.iyPos,SPB_XTD9_XGM/DOOCS/MAIN/current.bottom.output,SPB_XTD9_XGM/DOOCS/MAIN/current.bottom.rangeCode,SPB_XTD9_XGM/DOOCS/MAIN/current.left.output,SPB_XTD9_XGM/DOOCS/MAIN/current.left.rangeCode,SPB_XTD9_XGM/DOOCS/MAIN/current.right.output,SPB_XTD9_XGM/DOOCS/MAIN/current.right.rangeCode,SPB_XTD9_XGM/DOOCS/MAIN/current.top.output,SPB_XTD9_XGM/DOOCS/MAIN/current.top.rangeCode,...,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.crossUsed,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.gammaUsed,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.gmdError,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.nummberOfBrunches,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.photonFlux,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.pressure,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.temperature,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.usedGasType,SA1_XTD2_XGM/DOOCS/MAIN/pulseEnergy.wavelengthUsed,SA1_XTD2_XGM/DOOCS/MAIN/signalAdaption.dig
trainId,,,,,,,,,,,,,,,,,,,,,
10000,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0
10001,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0
10002,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0
10003,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0
10004,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0


## Run directories

We can do the same things with a run directory holding a collection of files.

Another dummy example:

In [12]:
!ls fxe_example_run/

RAW-R0445-LPD00-S00000.h5  RAW-R0445-LPD06-S00000.h5  RAW-R0445-LPD12-S00000.h5
RAW-R0445-LPD01-S00000.h5  RAW-R0445-LPD07-S00000.h5  RAW-R0445-LPD13-S00000.h5
RAW-R0445-LPD02-S00000.h5  RAW-R0445-LPD08-S00000.h5  RAW-R0445-LPD14-S00000.h5
RAW-R0445-LPD03-S00000.h5  RAW-R0445-LPD09-S00000.h5  RAW-R0445-LPD15-S00000.h5
RAW-R0445-LPD04-S00000.h5  RAW-R0445-LPD10-S00000.h5  RAW-R0450-DA01-S00000.h5
RAW-R0445-LPD05-S00000.h5  RAW-R0445-LPD11-S00000.h5


In [13]:
from karabo_data import RunDirectory
run = RunDirectory('fxe_example_run/')

In [14]:
run.files

In [15]:
run.control_devices

{'FXE_XAD_GEC/CAM/CAMERA',
 'SA1_XTD2_XGM/DOOCS/MAIN',
 'SPB_XTD9_XGM/DOOCS/MAIN'}

In [16]:
run.instrument_device_channels

{'FXE_DET_LPD1M-1/DET/0CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/10CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/11CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/12CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/13CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/14CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/15CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/1CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/2CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/3CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/4CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/5CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/6CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/7CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/8CH0:xtdf',
 'FXE_DET_LPD1M-1/DET/9CH0:xtdf',
 'FXE_XAD_GEC/CAM/CAMERA:daqOutput',
 'SA1_XTD2_XGM/DOOCS/MAIN:output',
 'SPB_XTD9_XGM/DOOCS/MAIN:output'}

In [17]:
print(run.train_ids[:10])

[10000, 10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009]


This collects data from across files, including detector data:

In [18]:
for tid, data in run.trains():
    print("Processing train", tid)
    print("Detctor data module 0 shape:", data['FXE_DET_LPD1M-1/DET/0CH0:xtdf']['image.data'].shape)

    break

Processing train 10000
Detctor data module 0 shape: (128, 1, 256, 256)


In [19]:
tid, data = run.train_from_id(10005)
tid, data = run.train_from_index(5)

It also chains sequence files, which contain successive data from the same source.

In [20]:
ixPos = run.get_series('SA1_XTD2_XGM/DOOCS/MAIN', 'beamPosition.ixPos.value')
ixPos.tail(10)

trainId
10470    0.0
10471    0.0
10472    0.0
10473    0.0
10474    0.0
10475    0.0
10476    0.0
10477    0.0
10478    0.0
10479    0.0
Name: SA1_XTD2_XGM/DOOCS/MAIN/beamPosition.ixPos, dtype: float32

In [21]:
# Select interesting data fields with glob syntax:
# [abc] = one of these characters, ? = any character, * = any sequence
run.get_dataframe(fields=[("*_XGM/*", "*.i[xy]Pos")])

,SPB_XTD9_XGM/DOOCS/MAIN/beamPosition.ixPos,SPB_XTD9_XGM/DOOCS/MAIN/beamPosition.iyPos,SA1_XTD2_XGM/DOOCS/MAIN/beamPosition.ixPos,SA1_XTD2_XGM/DOOCS/MAIN/beamPosition.iyPos
trainId,,,,
10000,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0
10002,0.0,0.0,0.0,0.0
10003,0.0,0.0,0.0,0.0
10004,0.0,0.0,0.0,0.0
10005,0.0,0.0,0.0,0.0
10006,0.0,0.0,0.0,0.0
10007,0.0,0.0,0.0,0.0
10008,0.0,0.0,0.0,0.0


## General information

In [22]:
run.info()

# of trains:    480
Duration:       0:00:47.900000
First train ID: 10000
Last train ID:  10479

16 detector modules (LPD)
  e.g. module LPD00 : 256 x 256 pixels
  128 frames per train, 61440.0 total frames

3 instrument devices (excluding detectors):
  - FXE_XAD_GEC/CAM/CAMERA:daqOutput
  - SA1_XTD2_XGM/DOOCS/MAIN:output
  - SPB_XTD9_XGM/DOOCS/MAIN:output

3 control devices:
  - FXE_XAD_GEC/CAM/CAMERA
  - SA1_XTD2_XGM/DOOCS/MAIN
  - SPB_XTD9_XGM/DOOCS/MAIN



In [23]:
H5File('fxe_example_run/RAW-R0445-LPD00-S00000.h5').detector_info()

{'dims': (256, 256), 'frames_per_train': 128, 'total_frames': 61440}

In [24]:
!lsxfel fxe_example_run/RAW-R0445-LPD00-S00000.h5

RAW-R0445-LPD00-S00000.h5 : Raw detector data from LPD module 00
480 trains

256 × 256 pixels
128 frames per train, 61440 total


In [25]:
!lsxfel fxe_example_run/RAW-R0450-DA01-S00000.h5

RAW-R0450-DA01-S00000.h5 : Aggregated data
480 trains

3 instrument devices
  -  FXE_XAD_GEC/CAM/CAMERA:daqOutput
  -  SA1_XTD2_XGM/DOOCS/MAIN:output
  -  SPB_XTD9_XGM/DOOCS/MAIN:output

3 control devices
  -  FXE_XAD_GEC/CAM/CAMERA
  -  SA1_XTD2_XGM/DOOCS/MAIN
  -  SPB_XTD9_XGM/DOOCS/MAIN



In [26]:
!lsxfel fxe_example_run

fxe_example_run : Run directory

# of trains:    480
Duration:       0:00:47.900000
First train ID: 10000
Last train ID:  10479

16 detector modules (LPD)
  e.g. module LPD00 : 256 × 256 pixels
  128 frames per train, 61440.0 total frames

3 instrument devices (excluding detectors):
  - FXE_XAD_GEC/CAM/CAMERA:daqOutput
  - SA1_XTD2_XGM/DOOCS/MAIN:output
  - SPB_XTD9_XGM/DOOCS/MAIN:output

3 control devices:
  - FXE_XAD_GEC/CAM/CAMERA
  - SA1_XTD2_XGM/DOOCS/MAIN
  - SPB_XTD9_XGM/DOOCS/MAIN

